# BYOC training for paddleOCR

In [ ]:
!cat Dockerfile

In [ ]:
#first build docker
!sh build_and_push.sh

## step1: upload data

In [ ]:
import sagemaker as sage
import boto3
from time import gmtime, strftime
from sagemaker import get_execution_role

#sess = sage.Session()
sess = sage.LocalSession()

WORK_DIRECTORY = "./input/data"

# S3 prefix
prefix = "DEMO-paddle-byo"

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## step2: local train (to debug)

In [ ]:
# local mode
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/paddle".format(account, region)
sess = sage.LocalSession()

hyperparameters = {"epoch_num": 10,
                  "print_batch_step":5,
                  "save_epoch_step":30,
                  'pretrained_model':'/opt/program/pretrain/ch_ppocr_mobile_v2.0_rec_train/best_accuracy'}

train = sage.estimator.Estimator(
    image,
    role,
    instance_count = 1,
    sagemaker_session=sess,
    instance_type='local_gpu',
    hyperparameters=hyperparameters,
)


train.fit(data_location)

## optional: create training job for training

In [ ]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/paddle".format(account, region)

sess = sage.Session()
hyperparameters = {"epoch_num": 10,
                  "print_batch_step":5,
                  "save_epoch_step":3,
                  'pretrained_model':'/opt/program/pretrain/ch_ppocr_mobile_v2.0_rec_train/best_accuracy'}

train = sage.estimator.Estimator(
    image,
    role,
    instance_count = 1,
    sagemaker_session=sess,
    instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters,
)


train.fit(data_location)

 # Inference

A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the topic mixture representing a given document.This section involves several steps,
Create Model 
- Create model for the training outputCreate Endpoint Configuration 
- Create a configuration defining an endpoint.Create Endpoint 
- Use the configuration to create an inference endpoint.Perform Inference 
- Perform inference on some input data using the endpoint.

## deploy model
we now create a SageMaker Model from the training output. Using the model we can create an Endpoint Configuration.


In [ ]:
%%time

import boto3
from time import gmtime, strftime

sage = boto3.Session().client(service_name='sagemaker') 

from sagemaker import get_execution_role
role = get_execution_role()

model_name="paddle-v0"
print(model_name)

info = sage.describe_training_job(TrainingJobName=train.latest_training_job.name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']

#model_data = train.model_data
#print(model_data)

# hosting_image = "847380964353.dkr.ecr.us-west-2.amazonaws.com/paddle"
primary_container = {
    'Image': image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

## Create Endpoint Configuration
#At launch, we will support configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, customers create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way.In addition, the endpoint configuration describes the instance type required for model deployment, and at launch will describe the autoscaling configuration.


In [ ]:
from time import gmtime, strftime
import time 

job_name_prefix = "paddle"

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.g4dn.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

##  Create Endpoint
Lastly, the customer creates the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [ ]:
%%time
import time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sage.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

 If you see the message, Endpoint creation ended with ```EndpointStatus = InService``` then congratulations! You now have a functioning inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console.We will finally create a runtime object from which we can invoke the endpoint.

#  Perform Inference

In [ ]:
%%time
import boto3
from botocore.config import Config
from sagemaker.session import Session

config = Config(
    read_timeout=120,
    retries={
        'max_attempts': 0
    }
)

from boto3.session import Session
import json

bucket = sess.default_bucket()
image_uri = 'DEMO-paddle-byo/test/1.jpg'
test_data = {
    'bucket' : bucket,
    'image_uri' : image_uri,
    'content_type': "application/json",
}
payload = json.dumps(test_data)
print(payload)

sagemaker_runtime_client = boto3.client('sagemaker-runtime', config=config)
session = Session(sagemaker_runtime_client)

#     runtime = session.client("runtime.sagemaker",config=config)
response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload)

result = json.loads(response["Body"].read())
print (result)

# Clean up
When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [ ]:
print(endpoint_name)
sage.delete_endpoint(EndpointName=endpoint_name)
sage.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sage.delete_model(ModelName=model_name)

In [ ]:
endpoint_name